In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # 同じフォルダの .env を読む

print("OPENAI:", os.getenv("OPENAI_API_KEY") is not None)
print("TAVILY:", os.getenv("TAVILY_API_KEY") is not None)


OPENAI: True
TAVILY: True


In [4]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

# .env を同じフォルダから読み込む
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Tavily クライアント生成
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# モデル名
MODEL_NAME = "gpt-4o-mini"


In [5]:
from tavily import TavilyClient
print("OK")


OK


In [6]:
# Tavily 検索結果を返す関数（最新版対応）
def get_search_result(question):
    # すでに作った tavily クライアントを使う
    response = tavily.search(query=question)

    # 結果を辞書のまま返す（JSON文字列にしない）
    return {
        "result": response["results"]
    }


In [7]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
print(ret)


{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/', 'title': '東京駅(東京都)周辺のイベント', 'content': '東京駅(東京都)周辺で開催されるイベント情報27件をお届けします。今日開催されているイベントはもちろん、週末の「どこ行こう」に役立つ情報が満載！', 'score': 0.99966204, 'raw_content': None}, {'url': 'https://www.enjoytokyo.jp/event/list/sta200101/its04/', 'title': '今日行ける！東京駅周辺のおすすめイベント', 'content': '「東京駅周辺 今日」の開催中または開催予定のイベントを紹介。おでかけに詳しい編集部がセレクトしました。気になるイベントは開催期間や開催場所、最寄駅、地図、', 'score': 0.9995827, 'raw_content': None}, {'url': 'https://iko-yo.net/events?prefecture_ids%5B%5D=13&tags%5B%5D=%E6%9D%B1%E4%BA%AC%E9%A7%85&term=1', 'title': '東京都 東京駅の 子ども・親子おでかけ 今日のイベント情報', 'content': '東京都（東京駅）のイベント[今日]2025年12月13日(土) ; 東京都の注目イベント. 理想の家を描くワークショップを開催します！ 東京都中央区 ; 群馬県の注目スポット. スキーも雪 ...See more', 'score': 0.9994116, 'raw_content': None}, {'url': 'https://ekitan.com/event/station-2590', 'title': '東京駅周辺のイベント', 'content': '東京駅周辺のイベント一覧。日付、料金等を指定しての絞込みも。おでかけに役立つ情報を案内。', 'score': 0.99916387, 'raw_content': None}, {'url': 'https://www.jalan.net/event/130000/1

In [8]:
# ツール定義（SDK v2 対応版）
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        {
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近1ヶ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "検索したい質問文（例：'東京駅のイベントを教えて'）"
                        }
                    },
                    "required": ["question"]
                }
            }
        }
    ]


In [9]:
tools = define_tools()
print(tools)



------define_tools(ツール定義)------
[{'type': 'function', 'function': {'name': 'get_search_result', 'description': '最近1ヶ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': "検索したい質問文（例：'東京駅のイベントを教えて'）"}}, 'required': ['question']}}}]


In [10]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question}
        ],
        tools=tools,
        tool_choice="auto",  # モデルが必要ならツールを呼ぶ
    )
    return response


In [11]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)
tavily = TavilyClient(api_key=TAVILY_API_KEY)

MODEL_NAME = "gpt-4o-mini"


In [12]:
tools = define_tools()
response = ask_question("東京駅のイベントを教えて", tools)
print(response)


------define_tools(ツール定義)------
ChatCompletion(id='chatcmpl-CmH4COFVo9Pt98hAxV3DTWqcBOQeB', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_1G74l9YrsZZGdhJvEq8DZCr6', function=Function(arguments='{"question":"東京駅のイベントを教えて"}', name='get_search_result'), type='function')]))], created=1765621384, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_aa07c96156', usage=CompletionUsage(completion_tokens=21, prompt_tokens=92, total_tokens=113, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [13]:
print("load_dotenv:", 'load_dotenv' in globals())
print("client:", 'client' in globals())
print("tavily:", 'tavily' in globals())
print("define_tools:", 'define_tools' in globals())
print("ask_question:", 'ask_question' in globals())


load_dotenv: True
client: True
tavily: True
define_tools: True
ask_question: True


In [14]:
# ツール呼び出しが必要な場合の処理を行う関数（SDK v2 対応）
def handle_tool_call(response, question):
    # ツールコールの取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name

    # 引数（JSON文字列） → Python dict に変換
    arguments = json.loads(tool.function.arguments)

    # 関数の実行（get_search_result など）
    function_response = globals()[function_name](**arguments)

    # ツールの実行結果を JSON 文字列に変換（OpenAI に渡す形式）
    tool_content = json.dumps(function_response, ensure_ascii=False)

    # messages にツール実行結果を追加してモデルに再問い合わせ
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,  # tool_call の入った元のメッセージ
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": tool_content,  # ★ dict ではなく JSON文字列
            },
        ],
    )

    return response_after_tool_call


In [15]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [16]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- **東京都**の面積は約2,194平方キロメートルです。
- **沖縄県**の面積は約2,271平方キロメートルです。

したがって、沖縄県の方が東京都よりも広いです。


In [17]:
# Tavily 検索結果を返す関数（最新版）
def get_search_result(question):
    # Tavily でWeb検索
    response = tavily.search(query=question)

    # 検索結果を dict で返す（JSON文字列ではない）
    return {
        "result": response["results"]
    }


In [18]:
"get_search_result" in globals()


True

In [19]:
tools = define_tools()

question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)


------define_tools(ツール定義)------
最近1ヶ月間の東京駅でのイベント情報は以下の通りです：

1. **東京駅 丸の内駅舎・駅前広場 歴史ガイドツアー**
   - **日時**: 2023年10月28日（土）11:00～、15:00～
   - **内容**: 東京駅丸の内駅舎と駅前広場の歴史を知るガイドツアーを開催。
   - **詳細**: [鉄道コムで確認する](https://www.tetsudo.com/event/45806/)

2. **TRAVELER'S FACTORY POP-UP with VICTORINOX EVENT**
   - **日時**: 2023年11月12日～14日
   - **内容**: 東京駅改札内のイベントスペース「スクエアゼロ」で、通常中目黒や京都で行っているイベントを東京駅でも開催。
   - **詳細**: [トラベラーズファクトリーで確認する](https://www.travelers-factory.com/topics/2023/tokyo-station-popup-2023/)

3. **秋のヘッドフォン祭 2023**
   - **日時**: 2023年10月28日（土）
   - **内容**: 新会場でのフル開催。
   - **詳細**: [フジヤエービックからのお知らせ](https://www.jas-audio.or.jp/jas_cms/wp-content/uploads/2023/10/pressrelease_231006.pdf)

4. **秋の味覚マルシェ in 丸の内トラストシティ**
   - **日時**: 2023年10月12日（木）、13日（金）11:00-17:30
   - **内容**: 秋の味覚が楽しめるマルシェイベント。
   - **詳細**: [森トラストのニュースで確認する](https://www.mori-trust.co.jp/news/2023/20231006/)

これらのイベントは、東京駅周辺での多様な体験を提供していますので、訪れる際は是非チェックしてみてください。


In [20]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")


------define_tools(ツール定義)------

---ご利用ありがとうございました！---


In [21]:
# 環境変数を読み込む
import os
from dotenv import load_dotenv

# Tavily クライアント
from tavily import TavilyClient

# .env を読み込む（search.ipynb と同じフォルダにある前提）
load_dotenv()

# Tavily APIキーを使ってクライアント作成
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

print("Tavily 準備 OK")


Tavily 準備 OK


In [43]:
# ニュース検索
query = (
    "最近1週間のAIニュース 主要 3件 "
    "(site:nhk.or.jp OR site:reuters.com OR site:itmedia.co.jp OR site:ascii.jp "
    "OR (site:nikkei.com inurl:/article/)) "
    "-compass -theme -topics -word -tag -category -archive -ranking"
)


result = tavily.search(query=query, max_results=5)


print(result)


{'query': '最近1週間のAIニュース 主要 3件 ( OR OR OR OR ( inurl:/article/)) -compass -theme -topics -word -tag -category -archive -ranking', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '「AI」を日経が解説・最新ニュース一覧 - 日本経済新聞', 'url': 'https://www.nikkei.com/topics/24032501', 'content': '有料会員の方のみご利用になれます。気になる連載・コラム・キーワードをフォローすると、「Myニュース」でまとめよみができます。. # AI. ## 韓国、半導体受託生産の工場建設を検討\u3000官民で4800億円投資. 【ソウル=松浦奈美】韓国政府は10日、官民で半導体ファウンドリー（受託生産）工場を建設する計画を明らかにした。投資額は4兆5000億ウォン（約4800億円）になる。人工知能（AI）分野の競争力を高めるため、多様な半導体の国産化を推し進める。 政府が10日に半導体に関する政策を公表した。サムスン電子やSKハイニックスが手掛ける記憶用のメモリー半導体では圧倒的な世界シェアを持つ。だが演算や制御などに…. ## 対Google非常事態のChatGPT、伏兵はメタの「アボカド｣\u3000新AI投入か. NVIDIAの半導体･H200、対中輸出は「敵塩」\u3000規制緩和に米でも反発. ## NVIDIAの半導体･H200、対中輸出は「敵塩」\u3000規制緩和に米でも反発. ## OpenAI、SlackのCEO引き抜き\u3000法人向け事業の幹部に. ## 中小企業へのAI導入で協力\u3000G7産業・デジタル・技術相会合. ## キリンHDの「AI役員」評価上々\u30001議案に60の意見や論点提示. ## 音声AI新興のRecho、3億円調達\u3000金融のコールセンター業務代替へ. ## 前例踏襲するだけの人材は不要\u3000超知能は詰め込み教育をひっくり返す. ## ウーバーが日本で投資拡大\u30005年で3100億円超、CEOが表明. 

In [46]:
search_text = ""

for item in result["results"]:
    search_text += f"タイトル: {item['title']}\n"
    search_text += f"概要: {item.get('content', '')}\n"
    search_text += f"URL: {item['url']}\n\n"

print(search_text)


タイトル: 「AI」を日経が解説・最新ニュース一覧 - 日本経済新聞
概要: 有料会員の方のみご利用になれます。気になる連載・コラム・キーワードをフォローすると、「Myニュース」でまとめよみができます。. # AI. ## 韓国、半導体受託生産の工場建設を検討　官民で4800億円投資. 【ソウル=松浦奈美】韓国政府は10日、官民で半導体ファウンドリー（受託生産）工場を建設する計画を明らかにした。投資額は4兆5000億ウォン（約4800億円）になる。人工知能（AI）分野の競争力を高めるため、多様な半導体の国産化を推し進める。 政府が10日に半導体に関する政策を公表した。サムスン電子やSKハイニックスが手掛ける記憶用のメモリー半導体では圧倒的な世界シェアを持つ。だが演算や制御などに…. ## 対Google非常事態のChatGPT、伏兵はメタの「アボカド｣　新AI投入か. NVIDIAの半導体･H200、対中輸出は「敵塩」　規制緩和に米でも反発. ## NVIDIAの半導体･H200、対中輸出は「敵塩」　規制緩和に米でも反発. ## OpenAI、SlackのCEO引き抜き　法人向け事業の幹部に. ## 中小企業へのAI導入で協力　G7産業・デジタル・技術相会合. ## キリンHDの「AI役員」評価上々　1議案に60の意見や論点提示. ## 音声AI新興のRecho、3億円調達　金融のコールセンター業務代替へ. ## 前例踏襲するだけの人材は不要　超知能は詰め込み教育をひっくり返す. ## ウーバーが日本で投資拡大　5年で3100億円超、CEOが表明. ## オープンAI「消費者と法人向け、比率均等に」　企業導入に自信. ## スタートアップ、ドバイが結節点に　投資家交流に日本勢も参画. ## OpenAIに「三菱UFJ特別チーム」　AI新興、客先常駐型モデルを採用. ## クラウド経営管理のログラス、AIで投資家との対話支援. ## AI小説4000字、プロンプトは20万字　「1億総クリエイター」時代の創作. 大学の6割、教育にAI　入試で活用例も「不正」警戒で二の足なお. ## 大学の6割、教育にAI　入試で活用例も「不正」警戒で二の足なお. ## AIエージェントの将来性高まる　日経BP「トレンドマップ2025下半期」. ## リコー、低コストの生成AI入門モデル

In [47]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
print("OpenAI 準備 OK")



OpenAI 準備 OK


In [48]:
print("search_text exists:", "search_text" in globals())
print("client exists:", "client" in globals())


search_text exists: True
client exists: True


In [49]:
try:
    MODEL_NAME = "gpt-4.1-mini"

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "あなたは最新ニュースを正確に要約するアシスタントです。推測せず、検索結果にある情報だけで答えてください。"},
            {"role": "user", "content": f"【検索結果】\n{search_text}\n\n上の情報だけを根拠に、重要ニュースを3つ要約して。各: タイトル/要約/URL。"}
        ],
    )
    print(response.choices[0].message.content)

except Exception as e:
    print("ERROR TYPE:", type(e))
    print("ERROR:", e)


1. タイトル: 韓国、半導体受託生産工場を官民で建設へ　約4800億円投資  
要約: 韓国政府は半導体ファウンドリー（受託生産）工場の建設計画を発表し、官民合わせて約4兆5000億ウォン（約4800億円）を投資する。人工知能（AI）分野の競争力強化を狙い、半導体の国産化を推進する方針。  
URL: https://www.nikkei.com/topics/24032501

2. タイトル: NTTドコモビジネスがAIエージェントで特許明細書作成を自動化  
要約: NTTドコモビジネスは2025年6月からAIエージェントを用いた知財文書作成支援サービスを開始。これにより特許出願に関わる煩雑な作業の効率化を図り、生産性向上が期待されている。  
URL: https://xtech.nikkei.com/theme/ai/

3. タイトル: 欧州委、グーグルのAIモデル訓練に関し独禁法違反で調査開始  
要約: 欧州連合（ＥＵ）欧州委員会は、米アルファベット傘下のグーグルがオンラインコンテンツをAIモデルの訓練に使用している件について、独占禁止法違反の疑いで調査を開始した。  
URL: https://jp.reuters.com/business/tech-ai/


In [50]:
MODEL_NAME = "gpt-4.1-mini"

prompt = f"""
以下はWeb検索で取得した最新ニュース情報です。
この情報だけを根拠にして、重要なニュースを3つ選び、日本語で要約してください。

【出力ルール】
- 3件
- 各ニュースは「タイトル（30文字以内）」「要約（120文字以内）」「URL」の3行
- 書式は必ず以下：

1)
タイトル:
要約:
URL:

2)
タイトル:
要約:
URL:

3)
タイトル:
要約:
URL:

【検索結果】
{search_text}
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "あなたは最新ニュースを正確に要約するアシスタントです。推測せず、検索結果にある情報だけで答えてください。"},
        {"role": "user", "content": prompt}
    ],
)

print(response.choices[0].message.content)


1)
タイトル: 韓国、半導体受託生産工場へ4800億円投資
要約: 韓国政府はAI競争力強化のため、官民で4800億円を投じ半導体ファウンドリー工場建設を検討。多様な半導体国産化を推進し、サムスンやSKハイニックスのメモリー以外の分野を強化する。
URL: https://www.nikkei.com/topics/24032501

2)
タイトル: 大成建設、AIで施工計画書作成を効率化
要約: 大成建設は生成AIを活用し、土木工事の施工計画書を約10分で自動生成できるシステムを開発。文書や画像、音声を同時解析し、作業時間を85％削減可能とする。
URL: https://xtech.nikkei.com/theme/ai/

3)
タイトル: 欧州委、グーグルのAIモデル訓練で独禁調査
要約: EU欧州委員会は米アルファベット傘下のグーグルがオンラインコンテンツをAIモデル訓練に利用している点について独占禁止法違反の疑いで調査を開始した。
URL: https://jp.reuters.com/business/tech-ai/


In [52]:
search_text = ""

for item in result["results"]:
    search_text += f"タイトル: {item.get('title','')}\n"
    search_text += f"概要: {item.get('content','')}\n"
    search_text += f"URL: {item.get('url','')}\n\n"

print(search_text)




タイトル: 「AI」を日経が解説・最新ニュース一覧 - 日本経済新聞
概要: 有料会員の方のみご利用になれます。気になる連載・コラム・キーワードをフォローすると、「Myニュース」でまとめよみができます。. # AI. ## 韓国、半導体受託生産の工場建設を検討　官民で4800億円投資. 【ソウル=松浦奈美】韓国政府は10日、官民で半導体ファウンドリー（受託生産）工場を建設する計画を明らかにした。投資額は4兆5000億ウォン（約4800億円）になる。人工知能（AI）分野の競争力を高めるため、多様な半導体の国産化を推し進める。 政府が10日に半導体に関する政策を公表した。サムスン電子やSKハイニックスが手掛ける記憶用のメモリー半導体では圧倒的な世界シェアを持つ。だが演算や制御などに…. ## 対Google非常事態のChatGPT、伏兵はメタの「アボカド｣　新AI投入か. NVIDIAの半導体･H200、対中輸出は「敵塩」　規制緩和に米でも反発. ## NVIDIAの半導体･H200、対中輸出は「敵塩」　規制緩和に米でも反発. ## OpenAI、SlackのCEO引き抜き　法人向け事業の幹部に. ## 中小企業へのAI導入で協力　G7産業・デジタル・技術相会合. ## キリンHDの「AI役員」評価上々　1議案に60の意見や論点提示. ## 音声AI新興のRecho、3億円調達　金融のコールセンター業務代替へ. ## 前例踏襲するだけの人材は不要　超知能は詰め込み教育をひっくり返す. ## ウーバーが日本で投資拡大　5年で3100億円超、CEOが表明. ## オープンAI「消費者と法人向け、比率均等に」　企業導入に自信. ## スタートアップ、ドバイが結節点に　投資家交流に日本勢も参画. ## OpenAIに「三菱UFJ特別チーム」　AI新興、客先常駐型モデルを採用. ## クラウド経営管理のログラス、AIで投資家との対話支援. ## AI小説4000字、プロンプトは20万字　「1億総クリエイター」時代の創作. 大学の6割、教育にAI　入試で活用例も「不正」警戒で二の足なお. ## 大学の6割、教育にAI　入試で活用例も「不正」警戒で二の足なお. ## AIエージェントの将来性高まる　日経BP「トレンドマップ2025下半期」. ## リコー、低コストの生成AI入門モデル